In [1]:
!pip install openfermion --quiet

In [2]:
!pip install netket --quiet

In [6]:
!pip install qiskit --quiet

In [1]:
from openfermion.ops import FermionOperator
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_ground_state, get_sparse_operator
import numpy as np
import scipy
import scipy.linalg
import netket as nk

import json

import matplotlib.pyplot as plt
import time
from openfermion.utils import commutator, count_qubits, hermitian_conjugated
from scipy.sparse.linalg import eigsh,eigs
from openfermion.linalg import get_sparse_operator, get_ground_state
import openfermion as of
from openfermion.ops import QubitOperator


In [2]:
class get_model():
    def __init__(self,n_list,pbc,max_neighbor_order,model):
        self.n_list=n_list
        self.pbc=pbc
        self.model=model
        self.max_neighbor_order=max_neighbor_order
        self.conv_dic={'t-J':'Ham_tJ','Heisenberg':'Ham_Heisenberg'}

    def get_grid(self,draw=False):
        '''
        Generate 2D Lattice with nx and ny sites in x,y
        option to set periodic boundary conditions and choose the number of nearest neighbors
        '''
        dim=self.n_list
        NN=self.max_neighbor_order

        g=nk.graph.Grid(extent=dim, pbc=self.pbc)
        bv=g.basis_vectors
        g_lat=nk.graph.Lattice(bv,pbc=self.pbc,extent=dim,max_neighbor_order=NN)
        if draw:
            g_lat.draw()
        return g_lat
    
    

    
    def build_op(self,i,j,extra_qubit=True):
        if extra_qubit:
            Sx_str='Y'+str(0)+' '+'X'+str(i+1)+' '+'X'+str(j+1)
            Sy_str='Y'+str(0)+' '+'Y'+str(i+1)+' '+'Y'+str(j+1)
            Sz_str='Y'+str(0)+' '+'Z'+str(i+1)+' '+'Z'+str(j+1)
        else:
            Sx_str='X'+str(i)+' '+'X'+str(j)
            Sy_str='Y'+str(i)+' '+'Y'+str(j)
            Sz_str='Z'+str(i)+' '+'Z'+str(j)
            
            
        
        Sx=QubitOperator(Sx_str)
        
        Sy=QubitOperator(Sy_str)
        
        Sz=QubitOperator(Sz_str)
        
        return Sx+Sy+Sz
    
    
    def sisj_2d(self,g,extra_qubit,verbose=False):
        n_sites=len(g.sites)
        #potential=QubitOperator()
        k=0
        potential=0*QubitOperator('X0 X1')
        for gh in g.edges():
            i=gh[0]
            j=gh[1]
            op=self.build_op(i,j,extra_qubit=extra_qubit)
            op_rev=self.build_op(j,i,extra_qubit=extra_qubit)
            potential+=op+op_rev
            k+=1
        return potential
            

    def get_potential_sisj(self,g,extra_qubit,verbose=False):
        if g.ndim==2:
            potential=self.sisj_2d(g,extra_qubit)
            return potential
    

    
    def Ham_Heisenberg(self,g,extra_qubit):
        U=self.get_potential_sisj(g,extra_qubit)
        H=U
        return H
    
    def get_Hamiltonian(self,g,extra_qubit=True):
        print(self.model)
        if self.model in self.conv_dic.keys():
            H=eval('self.'+self.conv_dic[self.model])(g,extra_qubit)
            return H
        else:
            return print('error model not present')
        
    def sparse_diag(self,op,k=4,automatic_gs=False):
        '''
        input: OpenFermion Second quantized operator
        output: eigenvalues and eigenvectors
        '''
        sparse_op = get_sparse_operator(op)
        if automatic_gs:
            E_gs=get_ground_state(sparse_op)
            print('GS energy using OpenFermion built in',E_gs)
        w,v=eigsh(sparse_op,k=k,which='SA')
        print('lowest lying eigenvalues',np.sort(w))
        idx = np.argsort(w)
        w = w[idx]
        v = v[:,idx]
        return w,v,sparse_op
    


In [3]:
from scipy.linalg import cosm



from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_ground_state, get_sparse_operator
import numpy
import scipy
import scipy.linalg
from scipy.linalg import eigh





def rescal_ham(Ham,wa_min,wa_max,beta=1.,alpha=0.):
    num_qubits=Ham.num_qubits
    eig_gap=wa_max-wa_min
    Id=Id_op(num_qubits)
    new_ham_1=Ham.mul((beta-alpha)/eig_gap)
    new_ham_2=Id.mul(-(beta-alpha)*wa_min/eig_gap+alpha)
    new_ham=new_ham_1.add(new_ham_2)
    return new_ham

def process_ham(Ham,check_eigen=True):
    wa,v=eigh(Ham.to_matrix())
    wa_max=max(wa)
    wa_min=min(wa)
    print(wa_max,wa_min)
    Ham_resc=rescal_ham(Ham,wa_min,wa_max)
    new_wa,new_v=eigh(Ham_resc.to_matrix())
    new_wa_min=min(new_wa)
    if check_eigen:
        print('new min eigenvalue',new_wa_min,'; new max eigenvalue',max(new_wa))
    #Ham_algo=build_ham_cirac(Ham_resc,new_wa_min)
    return Ham_resc,new_wa,new_v


In [4]:
def angular_momentum(g):
    Sx=QubitOperator()
    Sy=QubitOperator()
    Sz=QubitOperator()
    for i in g.nodes():
        valx='X'+str(i)
        valy='Y'+str(i)
        valz='Z'+str(i)
        Sx+=QubitOperator(valx,0.5)
        Sy+=QubitOperator(valy,0.5)
        Sz+=QubitOperator(valz,0.5)
    return Sx,Sy,Sz

from openfermion.linalg import eigenspectrum
def Id_op_of(num_qubits):
    val='I'
    for i in range(num_qubits):
        val+=' '+val
    return QubitOperator(val)

def rescal_ham_of(Ham,num_qubits,wa_min,wa_max,beta=1.,alpha=0.):
    eig_gap=wa_max-wa_min
    Id=QubitOperator.identity()#Id_op_of(num_qubits)
    new_ham_1=Ham*((beta-alpha)/eig_gap)
    new_ham_2=Id
    const=(-(beta-alpha)*wa_min/eig_gap+alpha)
    new_ham=new_ham_1+new_ham_2*const
    return new_ham

In [5]:
nx=3
ny=3
N=[nx,ny]
max_neighbors=1
M=get_model(N,True,max_neighbors,'Heisenberg')
g=M.get_grid()
H=M.get_Hamiltonian(g,extra_qubit=False)

Heisenberg


In [19]:
print_H=False
if print_H:
    print(H)
    

In [6]:
Sx,Sy,Sz=angular_momentum(g)
S2=(Sx*Sx)+(Sy*Sy)+(Sz*Sz)
from openfermion.linalg import qubit_operator_sparse as qos

H_mat=qos(H)
S2_mat=qos(S2)

In [7]:
H_mat=qos(H)

In [8]:
from scipy.sparse.linalg import eigsh
wn,vn=eigsh(H_mat,k=10,which='SA')
wn_max,vn_max=eigsh(H_mat,k=3,which='LA')

In [9]:
-31.752014   +29.48912529

-2.2628887099999986

In [10]:
wn,wn_max

(array([-31.752014  , -31.752014  , -29.48912529, -31.752014  ,
        -29.48912529, -31.752014  , -31.752014  , -31.752014  ,
        -31.752014  , -31.752014  ]),
 array([36., 36., 36.]))

In [11]:
wa_min=wn[0]
wa_max=wn_max[0]

In [12]:
num_qubits=nx*ny
new_H=rescal_ham_of(H,num_qubits,wa_min,wa_max,beta=1.,alpha=0.)

In [13]:
new_H_mat=qos(new_H)

In [14]:
w,v=eigsh(new_H_mat,k=10,which='SA')
w_max,v_max=eigsh(new_H_mat,k=3,which='LA')

In [15]:
w

array([4.47677933e-15, 4.48296621e-15, 4.41517529e-15, 4.38113857e-15,
       4.39239896e-15, 4.38465655e-15, 4.48109599e-15, 4.43900787e-15,
       3.33995784e-02, 3.33995784e-02])

In [16]:
w2=list(np.sort(w))
for wh in w2[1:]:
    check=abs(wh-w2[0])
    if abs(check)>1e-4:
        test=check
        break
        
test

0.033399578444578665